# 5. Algorithmic question

**Disclamair**: I took and adapted some of the following coding ideas from https://www.geeksforgeeks.org/k-maximum-sums-non-overlapping-contiguous-sub-arrays/ and also from the discussions on
https://www.hackerrank.com/challenges/maximum-subarray-sum/problem.


Consult for managing back-to-back sequences of requests for appointments. A sequence of requests is of the form `[30, 40, 25, 50, 30, 20]` where each number is the time that the person who makes the appointment wants to spend. Aaccept some requests with a break between them. Two consecutive requests are not accepptable. 

For example, `[30, 50, 20]` is an acceptable solution (of duration 100), but `[30, 40, 50, 20]` is not, because 30 and 40 are two consecutive appointments. 

**Goal**: provide a schedule that maximizes the total length of the accepted appointments. Provide also:
- an algorithm that computes the acceptable solution with the longest possible duration;
- a program that given in input an instance in the form given above, gives the optimal solution

For example, in the previous instance, the optimal solution is `[40, 50, 20]`, of total duration 110.

## Formalization of the problem

Given an array of positive integers, find the maximum sum of all the subsequences with the constraint that no two numbers in the subsequences are adjacent in the array and return both the maximum sum and the subsequence(s) that realize the maximum sum. If $f=f(v)$ is the function we want to implement and $v=(30, 40, 25, 50, 30, 20)$, then we should have $f(v)=(40, 50, 20)$ with sum $s=110$, as in the example above.

**Algorithmic idea: Dynamic programming**. Given an array $v$, let $v^*[i]$ be the optimal solution using the elements with indices $0,..,i$. In order to have a recursive algorithm that terminates set $v^*[0] = v[0]$, and  $v^*[i] = \max(v^*[i - 1], v^*[i - 2] + v[i])$ for $i = 1, ..., n$ (where $n$ is the dimension of the array given in input). Clearly $v^*[n]$ is the solution we want and it is obteined in $O(n)$. We can then use another array to store which choice is made for each subproblem, and so recover the actual elements chosen.

The same idea can be used to solve a more general problem as shown in the examples at the end of this paragraph.

## Code

In [1]:
# allows to initialize dictionaries with a lambda function 
# and provides the default value for a nonexistent key.
#so a defaultdict will never raise a KeyError.
from collections import defaultdict

In [2]:
dd = defaultdict(lambda: -1)
prefix_sum = []
trace = []

In [3]:
def sub_array_sum(i, j):
    """
    Input: indexes i,j of an array v with i<j
    Output: v[i]+v[i+1]+...+v[j-1]+v[j]
    Remark: if i>j returns 0
    """
    if i == 0:
        return prefix_sum[j]
    return (prefix_sum[j] - prefix_sum[i - 1])

In [4]:
def maximum_sum(cur, v, k):
    """
    Input: current element cur, array v, positive integer k 
    Output: current maximum sum 
    Remark: this function allows also track the elements that realise the maximum sum.      
    """
    if cur >= len(v):
        return 0
    if dd[cur] != -1:
        return dd[cur]
    
    # use the following line when all the elements in the array are positive, 
    # else set s1 and s2 to -Infinity
    s1 = -1; s2 = -1
    
    # choose subarray starting at the current element "cur"
    if cur + k - 1 < len(v):
        # Remark: sub_array_sum(cur,cur)=0
        s1 = sub_array_sum(cur, cur + k - 1) + maximum_sum(cur + k + 1, v, k)
    
    # ignore subarray starting at "cur"
    s2 = maximum_sum(cur + 1, v, k)
    dd[cur] = max(s1, s2)
    
    if s1 >= s2:
        # keep track of the elements that realise the maximum sum
        trace[cur] = (True, cur + k + 1)
        return s1
    trace[cur] = (False, cur + 1)
    
    return s2

In [5]:
def sub_array(v, trace, k):
    """
    Input: array v, array trace, positive integer k 
    Output: optimal solution, i.e. optimal subarray
    Remark: this function allows to return non-consecutive subarrays of size k 
            for every positive integer k, but in our problem only the case 
            k=1 is of interest.
    """
    i = 0
    subArrays = []
    for i in range(len(trace)):
        if trace[i][0]:
            subArrays.append(v[i : i + k])
        i = trace[i][1]

    return subArrays

In [6]:
def solution(v, k):
    """
    Input: array v, positive integer k 
    Output: optimal solution, i.e. optimal subarray(s)
    Remark: this function allows to return non-consecutive optimal subarray(s) of size k 
            for every positive integer k, but in our problem only the case 
            k=1 is of interest.
    """
    global dd, trace, prefix_sum
    dd = defaultdict(lambda: -1)
    
    # initialization
    trace = [(False, 0)] * len(v)
    prefix_sum = [0] * len(v)
    prefix_sum[0] = v[0]
    
    for i in range(1,len(v)):
        prefix_sum[i] += prefix_sum[i - 1] + v[i]
        
    print("Array :", v)
    print("Max sum: ", maximum_sum(0, v, k))
    print("Subarrays: ", sub_array(v, trace, k))

## Some examples

To solve the problem in question always choose $k=1$

In [7]:
solution([1,2,3,4,5], 1)

Array : [1, 2, 3, 4, 5]
Max sum:  9
Subarrays:  [[1], [3], [5]]


In [8]:
solution([30, 40, 25, 50, 30, 20], 1)

Array : [30, 40, 25, 50, 30, 20]
Max sum:  110
Subarrays:  [[40], [50], [30], [20]]


## Some examples of solution of a more general problem

To sole a generalized version of the problem take $k>1$, as shown below

In [9]:
solution([1,2,3,4,5], 2)

Array : [1, 2, 3, 4, 5]
Max sum:  12
Subarrays:  [[1, 2], [4, 5]]


In [10]:
solution([30, 40, 25, 50, 30, 20], 2)

Array : [30, 40, 25, 50, 30, 20]
Max sum:  150
Subarrays:  [[30, 40], [40, 25], [50, 30], [30, 20]]


In [11]:
solution([1,2,3,4,5], 3)

Array : [1, 2, 3, 4, 5]
Max sum:  12
Subarrays:  [[3, 4, 5]]


In [12]:
solution([30, 40, 25, 50, 30, 20], 3)

Array : [30, 40, 25, 50, 30, 20]
Max sum:  115
Subarrays:  [[40, 25, 50], [25, 50, 30], [50, 30, 20]]


## Problems with the code

In [13]:
# I don't know why the following happens happens...
# The problem seems to be with the function sub_array
# I've tryed to add a condition like the following
# if subArrays[i][0]-subArray[i-1][0] == v[i] - v[i-1] then do not append the elements starting qith v[i]
# but it seems it doesn't work... Any ideas?
solution([1,2,3,5,4],1)

Array : [1, 2, 3, 5, 4]
Max sum:  8
Subarrays:  [[1], [2], [3], [5], [4]]


In [14]:
solution([1,2,3,4,5],1)

Array : [1, 2, 3, 4, 5]
Max sum:  9
Subarrays:  [[1], [3], [5]]


In [15]:
f=lambda a:a and max([a[:1],a[:1]+f(a[2:]),f(a[1:])],key=sum)or a
for a, s in [([1,2,3,5,4], 1)]:
	print(f(a), sum(f(a)))

[1, 3, 4] 8


## Alternative solution

With immense surprise we have found that it is possible to solve the problem with just 3 lines of code! See https://codegolf.stackexchange.com/questions/183390/maximum-summed-subsequences-with-non-adjacent-items?answertab=active#tab-top for more deatils. 

Here it is the solution. 

In [16]:
v = [30, 40, 25, 50, 30, 20]
k = 1

In [17]:
f=lambda a:a and max([a[:1],a[:1]+f(a[2:]),f(a[1:])],key=sum)or a
for a, s in [(v, k)]:
	print(f(a), sum(f(a)))

[40, 50, 20] 110


**Credits**: Chas Brown https://codegolf.stackexchange.com/users/69880/chas-brown